In [1]:
%cd ../..

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [5]:
from price_predictor.data_preparation.split import split_train_test_valid
from price_predictor.data_preparation.feature_engineering import add_class_feature
from price_predictor.data_preparation.pick_or_drop_features import (
    pick_by_datatypes,
    drop_anonymous_features,
    drop_categorical_features,
    drop_categorical_except
)

In [6]:
from price_predictor.trainers.train_ensamble_model import train_random_forest, train_ada_boost
from price_predictor.evaluate import evaluate_model

In [7]:
data = pd.read_csv('data/cars.csv')
data.dropna(inplace=True)

In outlier data analysis I found out that most expensive car were either a van 
or SUV. Lets find out how much the body type affect the price

## Try numerical with body type only

In [8]:
df = drop_anonymous_features(data)
df = drop_categorical_except(df, ['body_type'])
df

,odometer_value,year_produced,engine_has_gas,engine_capacity,body_type,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed
0,190000,2010,False,2.5,universal,False,10900.00,False,9,13,16
1,290000,2002,False,3.0,universal,False,5000.00,True,12,54,83
2,402000,2001,False,2.5,suv,False,2800.00,True,4,72,151
3,10000,1999,False,3.0,sedan,False,9999.00,True,9,42,86
4,280000,2001,False,2.5,universal,False,2134.11,True,14,7,7
...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,sedan,False,2750.00,True,5,85,301
38527,321000,2004,False,2.2,hatchback,False,4800.00,True,4,20,317
38528,777957,2000,False,3.5,sedan,False,4300.00,False,3,63,369
38529,20000,2001,False,2.0,minivan,False,4000.00,True,7,156,490


In [9]:
df = pd.get_dummies(df)
df

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,...,body_type_hatchback,body_type_liftback,body_type_limousine,body_type_minibus,body_type_minivan,body_type_pickup,body_type_sedan,body_type_suv,body_type_universal,body_type_van
0,190000,2010,False,2.5,False,10900.00,False,9,13,16,...,0,0,0,0,0,0,0,0,1,0
1,290000,2002,False,3.0,False,5000.00,True,12,54,83,...,0,0,0,0,0,0,0,0,1,0
2,402000,2001,False,2.5,False,2800.00,True,4,72,151,...,0,0,0,0,0,0,0,1,0,0
3,10000,1999,False,3.0,False,9999.00,True,9,42,86,...,0,0,0,0,0,0,1,0,0,0
4,280000,2001,False,2.5,False,2134.11,True,14,7,7,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,301,...,0,0,0,0,0,0,1,0,0,0
38527,321000,2004,False,2.2,False,4800.00,True,4,20,317,...,1,0,0,0,0,0,0,0,0,0
38528,777957,2000,False,3.5,False,4300.00,False,3,63,369,...,0,0,0,0,0,0,1,0,0,0
38529,20000,2001,False,2.0,False,4000.00,True,7,156,490,...,0,0,0,0,1,0,0,0,0,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('price_usd', axis=1), df['price_usd'])

## Random Forest

In [13]:
rf = train_random_forest(X_train, y_train)
evaluate_model(rf, X_test, y_test)

{'rmse': 2416.743464058492,
 'mae': 1457.2447660646897,
 'r2': 0.8549143878962937}

## Conclusion: body types affects slightly the performance of our model